# Chapter 2 — An Array of Sequences

**Sections with code snippets in this chapter:**

* [List Comprehensions and Generator Expressions](#List-Comprehensions-and-Generator-Expressions)
* [Tuples Are Not Just Immutable Lists](#Tuples-Are-Not-Just-Immutable-Lists)
* [Unpacking sequences and iterables](#Unpacking-sequences-and-iterables)
* [Pattern Matching with Sequences](#Pattern-Matching-with-Sequences)
* [Slicing](#Slicing)
* [Using + and * with Sequences](#Using-+-and-*-with-Sequences)
* [Augmented Assignment with Sequences](#Augmented-Assignment-with-Sequences)
* [list.sort and the sorted Built-In Function](#list.sort-and-the-sorted-Built-In-Function)
* [When a List Is Not the Answer](#When-a-List-Is-Not-the-Answer)
* [Memory Views](#Memory-Views)
* [NumPy and SciPy](#NumPy-and-SciPy)
* [Deques and Other Queues](#Deques-and-Other-Queues)
* [Soapbox](#Soapbox)

## List Comprehensions and Generator Expressions

#### Example 2-1. Build a list of Unicode codepoints from a string

In [ ]:
symbols = '$¢£¥€¤'
codes = []

for symbol in symbols:
    codes.append(ord(symbol))

codes

#### Example 2-2. Build a list of Unicode codepoints from a string, using a listcomp

In [ ]:
symbols = '$¢£¥€¤'

codes = [ord(symbol) for symbol in symbols]

codes

#### Box: Listcomps No Longer Leak Their Variables

In [ ]:
x = 'ABC'
codes = [ord(x) for x in x]
x

In [ ]:
codes

In [ ]:
codes = [last := ord(c) for c in x]
last

#### Example 2-3. The same list built by a listcomp and a map/filter composition

In [ ]:
symbols = '$¢£¥€¤'
beyond_ascii = [ord(s) for s in symbols if ord(s) > 127]
beyond_ascii

In [ ]:
beyond_ascii = list(filter(lambda c: c > 127, map(ord, symbols)))
beyond_ascii

#### Example 2-4. Cartesian product using a list comprehension

In [ ]:
colors = ['black', 'white']
sizes = ['S', 'M', 'L']
tshirts = [(color, size) for color in colors for size in sizes]
tshirts

In [ ]:
for color in colors:
    for size in sizes:
        print((color, size))

In [ ]:
shirts = [(color, size) for size in sizes
          for color in colors]
tshirts

#### Example 2-5. Initializing a tuple and an array from a generator expression

In [ ]:
symbols = '$¢£¥€¤'
tuple(ord(symbol) for symbol in symbols)

In [ ]:
import array

array.array('I', (ord(symbol) for symbol in symbols))

#### Example 2-6. Cartesian product in a generator expression

In [ ]:
colors = ['black', 'white']
sizes = ['S', 'M', 'L']

for tshirt in ('%s %s' % (c, s) for c in colors for s in sizes):
    print(tshirt)

## Tuples Are Not Just Immutable Lists

#### Example 2-7. Tuples used as records

In [ ]:
lax_coordinates = (33.9425, -118.408056)
city, year, pop, chg, area = ('Tokyo', 2003, 32_450, 0.66, 8014)
traveler_ids = [('USA', '31195855'), ('BRA', 'CE342567'), ('ESP', 'XDA205856')]

for passport in sorted(traveler_ids):
    print('%s/%s' % passport)

In [ ]:
for country, _ in traveler_ids:
    print(country)

### Tuples as Immutable Lists

In [ ]:
a = (10, 'alpha', [1, 2])
b = (10, 'alpha', [1, 2])
a == b

In [ ]:
b[-1].append(99)
a == b

In [ ]:
b

In [ ]:
def fixed(o):
    try:
        hash(o)
    except TypeError:
        return False
    return True


tf = (10, 'alpha', (1, 2))  # Contains no mutable items
tm = (10, 'alpha', [1, 2])  # Contains a mutable item (list)
fixed(tf)

In [ ]:
fixed(tm)

## Unpacking sequences and iterables

In [ ]:
lax_coordinates = (33.9425, -118.408056)
latitude, longitude = lax_coordinates  # unpacking
latitude

In [ ]:
longitude

In [ ]:
divmod(20, 8)

In [ ]:
t = (20, 8)
divmod(*t)

In [ ]:
quotient, remainder = divmod(*t)
quotient, remainder

In [ ]:
import os

_, filename = os.path.split('/home/luciano/.ssh/id_rsa.pub')
filename

### Using * to grab excess items

In [ ]:
a, b, *rest = range(5)
a, b, rest

In [ ]:
a, b, *rest = range(3)
a, b, rest

In [ ]:
a, b, *rest = range(2)
a, b, rest

In [ ]:
a, *body, c, d = range(5)
a, body, c, d

In [ ]:
*head, b, c, d = range(5)
head, b, c, d

### Unpacking with * in function calls and sequence literals

In [ ]:
def fun(a, b, c, d, *rest):
    return a, b, c, d, rest


fun(*[1, 2], 3, *range(4, 7))

In [ ]:
*range(4), 4

In [ ]:
[*range(4), 4]

In [ ]:
{*range(4), 4, *(5, 6, 7)}

### Nested unpacking
#### Example 2-8. Unpacking nested tuples to access the longitude

[02-array-seq/metro_lat_lon.py](02-array-seq/metro_lat_lon.py)

## Pattern Matching with Sequences
#### Example 2-9. Method from an imaginary Robot class

In [ ]:

class Robot: 

  def beep(self, times, frequency):
      print(f'Beeped {times} times at {frequency} Hz')

  def rotate_neck(self, angle):
      print(f'Neck rotated at {angle} degrees')

  def set_color(self, ident, red, green, blue):
      print(f'LED {ident} set to {red}, {green}, {blue}')

  def set_brightness(self, ident, intensity):
      print(f'LED {ident} set to brightness {intensity}')

  
  def handle_command(self, message):
      match message:
          case ['BEEPER', frequency, times]:
              self.beep(times, frequency)
          case ['NECK', angle]:
              self.rotate_neck(angle)
          case ['LED', ident, intensity]:
              self.set_brightness(ident, intensity)
          case ['LED', ident, red, green, blue]:
              self.set_color(ident, red, green, blue)
          case _:
              raise RuntimeError(message)
          
robot = Robot()

# These should print
robot.handle_command(['BEEPER', 440, 3])
robot.handle_command(['NECK', 180])
robot.handle_command(['LED', 1, 0.2])
robot.handle_command(['LED', 1, 128, 128, 128])

# This should throw up
robot.handle_command(['AUTODESTRUCT'])

#### Example 2-10. Destructuring nested tuples—requires Python ≥ 3.10.
[02-array-seq/match_lat_lon.py](02-array-seq/match_lat_lon.py)

In [ ]:
metro_areas = [
    ('Tokyo', 'JP', 36.933, (35.689722, 139.691667)),
    ('Delhi NCR', 'IN', 21.935, (28.613889, 77.208889)),
    ('Mexico City', 'MX', 20.142, (19.433333, -99.133333)),
    ('New York-Newark', 'US', 20.104, (40.808611, -74.020386)),
    ('São Paulo', 'BR', 19.649, (-23.547778, -46.635833)),
]

def main():
    print(f'{"":15} | {"latitude":>9} | {"longitude":>9}')
    for record in metro_areas:
        match record:
            case [name, _, _, (lat, lon)] if lon <= 0:
                print(f'{name:15} | {lat:9.4f} | {lon:9.4f}')
main()

### Pattern Matching Sequences in an Interpreter
#### Example 2-11. Matching patterns without match/case.
[02-array-seq/lispy/py3.9/lis.py](02-array-seq/lispy/py3.9/lis.py)

#### Example 2-12. Pattern matching with match/case—requires Python ≥ 3.10.
[02-array-seq/lispy/py3.10/lis.py](02-array-seq/lispy/py3.10/lis.py)

## Slicing

### Why Slices and Range Exclude the Last Item

In [ ]:
l = [10, 20, 30, 40, 50, 60]

l[:2]  # split at 2

In [ ]:
l[2:]

In [ ]:
l[:3]  # split at 3

In [ ]:
l[3:]

### Slice Objects

In [ ]:
s = 'bicycle'
s[::3]

In [ ]:
s[::-1]

In [ ]:
s[::-2]

#### Example 2-13. Line items from a flat-file invoice

In [ ]:
invoice = """
0.....6.................................40........52...55........
1909 Pimoroni PiBrella                      $17.50    3    $52.50
1489 6mm Tactile Switch x20                  $4.95    2    $9.90
1510 Panavise Jr. - PV-201                  $28.00    1    $28.00
1601 PiTFT Mini Kit 320x240                 $34.95    1    $34.95
"""

SKU = slice(0, 6)
DESCRIPTION = slice(6, 40)
UNIT_PRICE = slice(40, 52)
QUANTITY = slice(52, 55)
ITEM_TOTAL = slice(55, None)

line_items = invoice.split('\n')[2:]

for item in line_items:
    print(item[UNIT_PRICE], item[DESCRIPTION])

### Assigning to Slices

In [ ]:
l = list(range(10))
l

In [ ]:
l[2:5] = [20, 30]
l

In [ ]:
del l[5:7]
l

In [ ]:
l[3::2] = [11, 22]
l

By design, this example raises an exception::

In [ ]:
try:
    l[2:5] = 100
except TypeError as e:
    print(repr(e))

In [ ]:
l[2:5] = [100]
l

## Using + and * with Sequences

In [ ]:
l = [1, 2, 3]
l * 5

In [ ]:
5 * 'abcd'

### Building Lists of Lists

#### Example 2-14. A list with three lists of length 3 can represent a tic-tac-toe board

In [ ]:
board = [['_'] * 3 for i in range(3)]
board

In [ ]:
board[1][2] = 'X'
board

#### Example 2-15. A list with three references to the same list is useless

In [ ]:
weird_board = [['_'] * 3] * 3
weird_board

In [ ]:
weird_board[1][2] = 'O'
weird_board

#### Explanation

In [ ]:
board = []
for i in range(3):
    row = ['_'] * 3
    board.append(row)
board

In [ ]:
board[2][0] = 'X'
board

## Augmented Assignment with Sequences

In [ ]:
l = [1, 2, 3]
idl = id(l)

In [ ]:
# NBVAL_IGNORE_OUTPUT
idl

In [ ]:
l *= 2
l

In [ ]:
id(l) == idl  # same list

In [ ]:
t = (1, 2, 3)
idt = id(t)

In [ ]:
# NBVAL_IGNORE_OUTPUT
idt

In [ ]:
t *= 2
id(t) == idt  # new tuple

### A += Assignment Puzzler
#### Example 2-16. A riddle

In [ ]:
t = (1, 2, [30, 40])
try:
    t[2] += [50, 60]
except TypeError as e:
    print(repr(e))

#### Example 2-17. The unexpected result: item t2 is changed and an exception is raised

In [ ]:
t

#### Example 2-18. Bytecode for the expression s[a] += b

In [ ]:
import dis

dis.dis('s[a] += b')

## list.sort and the sorted Built-In Function

In [ ]:
fruits = ['grape', 'raspberry', 'apple', 'banana']
sorted(fruits)

In [ ]:
fruits

In [ ]:
sorted(fruits, reverse=True)

In [ ]:
sorted(fruits, key=len)

In [ ]:
sorted(fruits, key=len, reverse=True)

In [ ]:
fruits

In [ ]:
fruits.sort()
fruits

## When a List Is Not the Answer

### Arrays

#### Example 2-19. Creating, saving, and loading a large array of floats

In [ ]:
from array import array
from random import random, seed
seed(10)  # Use seed to make the output consistent

floats = array('d', (random() for i in range(10 ** 7)))
floats[-1]

In [ ]:
with open('floats.bin', 'wb') as fp:
    floats.tofile(fp)

In [ ]:
floats2 = array('d')

with open('floats.bin', 'rb') as fp:
    floats2.fromfile(fp, 10 ** 7)

floats2[-1]

In [ ]:
floats2 == floats

### Memory Views

#### Example 2-20. Handling 6 bytes memory of as 1×6, 2×3, and 3×2 views

In [ ]:
octets = array('B', range(6))
m1 = memoryview(octets)
m1.tolist()

In [ ]:
m2 = m1.cast('B', [2, 3])
m2.tolist()

In [ ]:
m3 = m1.cast('B', [3, 2])
m3.tolist()

In [ ]:
m2[1,1] = 22
m3[1,1] = 33
octets

#### Example 2-21. Changing the value of an 16-bit integer array item by poking one of its bytes

In [ ]:
numbers = array('h', [-2, -1, 0, 1, 2])
memv = memoryview(numbers)
len(memv)

In [ ]:
memv[0]

In [ ]:
memv_oct = memv.cast('B')
memv_oct.tolist()

In [ ]:
memv_oct[5] = 4
numbers

### NumPy

#### Example 2-22. Basic operations with rows and columns in a numpy.ndarray

In [ ]:
import numpy as np

a = np.arange(12)
a

In [ ]:
type(a)

In [ ]:
a.shape

In [ ]:
a.shape = 3, 4
a

In [ ]:
a[2]

In [ ]:
a[2, 1]

In [ ]:
a[:, 1]

In [ ]:
a.transpose()

#### Example 2-22. Loading, saving, and vectorized operations

In [ ]:
with open('floats-1M-lines.txt', 'wt') as fp:
    for _ in range(1_000_000):
        fp.write(f'{random()}\n')

In [ ]:
floats = np.loadtxt('floats-1M-lines.txt')

In [ ]:
floats[-3:]

In [ ]:
floats *= .5
floats[-3:]

In [ ]:
from time import perf_counter as pc

t0 = pc()
floats /= 3
(pc() - t0) < 0.01

In [ ]:
np.save('floats-1M', floats)
floats2 = np.load('floats-1M.npy', 'r+')
floats2 *= 6

In [ ]:
floats2[-3:]

### Deques and Other Queues

#### Example 2-23. Working with a deque

In [ ]:
import collections

dq = collections.deque(range(10), maxlen=10)
dq

In [ ]:
dq.rotate(3)
dq

In [ ]:
dq.rotate(-4)
dq

In [ ]:
dq.appendleft(-1)
dq

In [ ]:
dq.extend([11, 22, 33])
dq

In [ ]:
dq.extendleft([10, 20, 30, 40])
dq

## Soapbox

### Mixed bag lists

In [ ]:
l = [28, 14, '28', 5, '9', '1', 0, 6, '23', 19]

In [ ]:
try:
    sorted(l)
except TypeError as e:
    print(repr(e))

### Key is Brilliant

In [ ]:
l = [28, 14, '28', 5, '9', '1', 0, 6, '23', 19]

sorted(l, key=int)

In [ ]:
sorted(l, key=str)